In [8]:
import sys
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib'
)
sys.path.insert(
    1,
    '/home/dtd/Documents/interpretable_machine_learning/Causal Inference/CEA')

import data_load
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math
import timeit
import evaluation as evl
import pandas as pd
import numpy as np
import tensorflow as tf

from dowhy import CausalModel
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import sem
from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt
from econml.dml import LinearDMLCateEstimator
from sklearn.linear_model import LassoCV
from econml.inference import BootstrapInference
from econml.dml import SparseLinearDMLCateEstimator
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.ensemble import GradientBoostingRegressor
from mlens.ensemble import SuperLearner
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [21]:
f = lambda a,b: 5*a + 4*b
a = tf.Variable(3., trainable = True)
b = tf.Variable(3., trainable = True)
true_effect = tf.constant(100.0)
threhold = 0.03
for i in tqdm(range(50000)):
    with tf.GradientTape() as tape:
        y = f(a,b)
        print(y)
        print(true_effect)
        loss = abs(y - true_effect)
        print(loss)
#         loss = tf.keras.losses.mean_absolute_error(true_effect, y)
        if loss.numpy() < threhold:
            print("Finish at loop: {}".format(i))
            break

        d_a, d_b = tape.gradient(loss, [a,b])
        a.assign_sub(d_a * 0.0001)
        b.assign_sub(d_b * 0.0001)
print(a,b,f(a,b))

  0%|          | 65/50000 [00:00<01:17, 642.16it/s]

tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(73.0, shape=(), dtype=float32)
tf.Tensor(27.0041, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.995895, shape=(), dtype=float32)
tf.Tensor(27.0082, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.9918, shape=(), dtype=float32)
tf.Tensor(27.0123, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.9877, shape=(), dtype=float32)
tf.Tensor(27.0164, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.9836, shape=(), dtype=float32)
tf.Tensor(27.0205, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.9795, shape=(), dtype=float32)
tf.Tensor(27.024601, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.9754, shape=(), dtype=float32)
tf.Tensor(27.028702, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.T

  0%|          | 207/50000 [00:00<01:14, 669.09it/s]

tf.Tensor(72.44239, shape=(), dtype=float32)
tf.Tensor(27.56171, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.43829, shape=(), dtype=float32)
tf.Tensor(27.565811, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.43419, shape=(), dtype=float32)
tf.Tensor(27.569912, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.430084, shape=(), dtype=float32)
tf.Tensor(27.574013, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.42599, shape=(), dtype=float32)
tf.Tensor(27.578112, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.42189, shape=(), dtype=float32)
tf.Tensor(27.582212, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.417786, shape=(), dtype=float32)
tf.Tensor(27.586311, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(72.41369, shape=(), dtype=float32)
tf.Tensor(27.590412, sh

  1%|          | 332/50000 [00:00<01:17, 644.37it/s]

...

tf.Tensor(1.6548843, shape=(), dtype=float32)
tf.Tensor(98.34921, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6507874, shape=(), dtype=float32)
tf.Tensor(98.35331, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6466904, shape=(), dtype=float32)
tf.Tensor(98.35741, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6425934, shape=(), dtype=float32)
tf.Tensor(98.3615, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6384964, shape=(), dtype=float32)
tf.Tensor(98.3656, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6343994, shape=(), dtype=float32)
tf.Tensor(98.3697, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6303024, shape=(), dtype=float32)
tf.Tensor(98.373795, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.6262054, shape=(), dtype=float32)
tf.Tensor(98.37789, shape

 35%|███▌      | 17653/50000 [00:25<00:49, 658.55it/s]

tf.Tensor(98.9146, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0854034, shape=(), dtype=float32)
tf.Tensor(98.91869, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0813065, shape=(), dtype=float32)
tf.Tensor(98.92279, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0772095, shape=(), dtype=float32)
tf.Tensor(98.92689, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0731125, shape=(), dtype=float32)
tf.Tensor(98.930984, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0690155, shape=(), dtype=float32)
tf.Tensor(98.93508, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0649185, shape=(), dtype=float32)
tf.Tensor(98.93918, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(1.0608215, shape=(), dtype=float32)
tf.Tensor(98.943275, shape=(), dtype=float32)
tf.Tensor(100.0, shape=

 36%|███▌      | 17800/50000 [00:26<00:47, 684.48it/s]

tf.Tensor(0.57328033, shape=(), dtype=float32)
tf.Tensor(99.43082, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.56918335, shape=(), dtype=float32)
tf.Tensor(99.43491, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.56508636, shape=(), dtype=float32)
tf.Tensor(99.43901, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.5609894, shape=(), dtype=float32)
tf.Tensor(99.44311, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.5568924, shape=(), dtype=float32)
tf.Tensor(99.447205, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.5527954, shape=(), dtype=float32)
tf.Tensor(99.4513, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.5486984, shape=(), dtype=float32)
tf.Tensor(99.4554, shape=(), dtype=float32)
tf.Tensor(100.0, shape=(), dtype=float32)
tf.Tensor(0.54460144, shape=(), dtype=float32)
tf.Tensor(99.459496,

In [44]:
f = lambda a,b: 5*a + 4*b
a = tf.Variable(3., trainable = True)
b = tf.Variable(3., trainable = True)
true_effect = tf.constant(100.0)
threhold = 0.03

input_ = [a,b]
# with tf.GradientTape() as tape:
#     tape.watch([a,b])
#     y = f(a,b)
#     loss = abs(y - true_effect)
#     #loss = tf.reduce_mean(input)
#     print(loss)

# opt = tf.keras.optimizers.Adam(learning_rate=1)
    


for i in range(100):
    with tf.GradientTape() as tape:
        tape.watch([a,b])
        y = f(a,b)
        loss = abs(y - true_effect)
        gradients = tape.gradient(loss, input_)
        opt = tf.keras.optimizers.Adam(learning_rate=1)
        print("Honey")
        print(gradients)
        print(input_)
        print("\n")
        opt.apply_gradients(zip(gradients, input_))
        break
        
print(input_, f(a,b).numpy())   


Honey
[<tf.Tensor: shape=(), dtype=float32, numpy=-5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=-4.0>]
[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>, <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=3.0>]




TypeError: Cannot iterate over a scalar tensor.